## Setup

In [ ]:
!pip install llama-index
!pip install python-dotenv

In [22]:
load_dotenv()
monday_api_token = os.getenv('MONDAY_API_KEY')

In [5]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex

## Basic

In [19]:
documents = SimpleDirectoryReader("./data").load_data()

In [24]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [27]:
response = query_engine.query("Why is a mouse when it spins?")
print(response)

I'm unable to provide a relevant answer to that query based on the context information provided.


## Chroma

In [ ]:
!pip install chromadb
!pip install llama-index-vector-stores-chroma

In [31]:
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

In [32]:
db = chromadb.PersistentClient(path="./chroma_db")

In [33]:
chroma_collection = db.get_or_create_collection("ragtest")

In [34]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create your index
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

In [36]:
query_engine = index.as_query_engine()
response = query_engine.query("what is hygiene?")
print(response)

Hygiene involves maintaining cleanliness and taking measures to prevent the spread of dirt, viruses, bacteria, and other contaminants. It includes practices such as proper handwashing, avoiding touching food directly with bare hands, using appropriate tools like tongs or gloves, and ensuring personal cleanliness to prevent cross-contamination and maintain food safety standards.


## Monday API

In [ ]:
#!pip install llama-index-readers-mondaydotcom

In [93]:
# Custom Monday API reader

from typing import Dict, List
import requests
import json
from llama_index.core.readers.base import BaseReader
from llama_index.core.schema import Document

class MondayReader(BaseReader):
    """monday.com reader. Reads board's data by a GraphQL query."""
    def __init__(self, api_key: str) -> None:
        """Initialize monday.com reader."""
        self.api_key = api_key
        self.api_url = "https://api.monday.com/v2"

    def _parse_item_values(self, cv) -> Dict[str, str]:
        return {"title": cv.get("column", {}).get("title", ""), "value": cv.get("text", "")}

    def _parse_assets(self, assets) -> List[Dict[str, str]]:
        return [{"name": asset.get("name", ""), "url": asset.get("url", "")} for asset in assets]

    def _parse_updates(self, updates) -> List[Dict[str, str]]:
        return [{"body": update.get("body", ""), "created_at": update.get("created_at", "")} for update in updates]

    def _parse_data(self, item) -> Dict[str, any]:
        return {
            "id": item["id"],
            "name": item["name"],
            "values": [self._parse_item_values(cv) for cv in item["column_values"]],
            "assets": self._parse_assets(item.get("assets", [])),
            "updates": self._parse_updates(item.get("updates", [])),
            "subitems": [self._parse_data(subitem) for subitem in item.get("subitems", [])]
        }

    def _perform_request(self, board_id: int, cursor: str = None) -> Dict[str, any]:
        headers = {"Authorization": self.api_key}
        query = """
            query($boardId: ID!, $cursor: String) {
                boards(ids: [$boardId]) {
                    name,
                    items_page(limit: 100, cursor: $cursor) {
                        cursor
                        items {
                            id,
                            name,
                            column_values {
                                column { title }
                                text
                            }
                            assets {
                                name
                                url
                            }
                            updates {
                                body
                                created_at
                            }
                            subitems {
                                id,
                                name,
                                column_values {
                                    column { title }
                                    text
                                }
                                assets {
                                    name
                                    url
                                }
                                updates {
                                    body
                                    created_at
                                }
                            }
                        }
                    }
                }
            }
        """
        variables = {"boardId": board_id, "cursor": cursor}
        data = {"query": query, "variables": variables}
        response = requests.post(url=self.api_url, json=data, headers=headers)
        return response.json()

    def _subitem_to_dict(self, subitem):
        return {
            "id": subitem["id"],
            "name": subitem["name"],
            "values": [{"title": v["title"], "value": v["value"]} for v in subitem["values"]],
            "assets": [{"name": a["name"], "url": a["url"]} for a in subitem["assets"]],
            "updates": [{"body": u["body"], "created_at": u["created_at"]} for u in subitem["updates"]]
        }

    def load_data(self, board_id: int) -> List[Document]:
        """Load board data by board_id."""
        all_items = []
        cursor = None
        while True:
            json_response = self._perform_request(board_id, cursor)
            
            if "errors" in json_response:
                raise Exception(f"API Error: {json_response['errors']}")
            
            if "data" not in json_response or "boards" not in json_response["data"]:
                raise Exception("Unexpected API response structure")
            board_data = json_response["data"]["boards"][0]
            items_page = board_data["items_page"]
            
            all_items.extend(items_page["items"])
            
            if not items_page["cursor"]:
                break
            
            cursor = items_page["cursor"]
        parsed_items = [self._parse_data(item) for item in all_items]
        
        result = []
        for item in parsed_items:
            text = f"name: {item['name']}"
            for item_value in item["values"]:
                if item_value["value"]:
                    text += f", {item_value['title']}: {item_value['value']}"
            
            if item["assets"]:
                text += "\nAssets:"
                for asset in item["assets"]:
                    text += f"\n- {asset['name']}: {asset['url']}"
            
            if item["updates"]:
                text += "\nUpdates:"
                for update in item["updates"]:
                    text += f"\n- {update['created_at']}: {update['body']}"
            
            if item["subitems"]:
                text += "\nSubitems:"
                for subitem in item["subitems"]:
                    text += f"\n- {subitem['name']}"
                    for subitem_value in subitem["values"]:
                        if subitem_value["value"]:
                            text += f", {subitem_value['title']}: {subitem_value['value']}"

            result.append(
                Document(
                    text=text,
                    extra_info={
                        "board_id": board_id,
                        "item_id": item["id"],
                        "subitems_count": len(item["subitems"]),
                        "subitems_json": json.dumps([self._subitem_to_dict(subitem) for subitem in item["subitems"]])
                    }
                )
            )
        
        return result

In [ ]:
# Monday API connection
board_id = 1564566045
reader = MondayReader(monday_api_token)
raw_response = reader._perform_request(board_id)
documents = reader.load_data(board_id)

In [108]:
# Database setup
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("monday")

In [109]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create your index
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
query_engine = index.as_query_engine()

In [122]:
response = query_engine.query("What are the key symptoms of food poisining?")
print(response)

Symptoms of food poisoning typically include nausea, vomiting, diarrhea, abdominal pain, fever, and sometimes headache.
